## Import external and `aiproteomics` packages

In [1]:
import tensorflow as tf
import numpy as np

import aiproteomics

2022-12-09 10:04:31.975808: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 10:04:31.975919: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load a ready-to-use dataset derived from the DeepDIA model

It's now possible to load (remote) proteomics datasets defined in the package, with all preprocessing carried out automatically.

In [ ]:
# ds, ds_info = tfds.load('aiproteomicshela1', as_supervised=True, with_info=True)


Let's look at the info for the 'hela1' dataset we have just fetched:

In [ ]:
# print(ds_info)

## Build a transformer model for retention time prediction

The `aiproteomics` package will also contains quick methods for obtaining common retention time models. Currently we can build a transformer for learning retention time.

In [ ]:
model = aiproteomics.rt.models.build_rt_transformer_model(
        num_layers = 6,                 # number of layers, paper = 6
        d_model = 512,
        num_heads = 8,                  # Number of attention heads, paper = 8
        d_ff = 2048,                    # Hidden layer size in feed forward network inside transformer, paper = 2048
        dropout_rate = 0.1,             #
        vocab_size = 22,                # number of aminoacids
        max_len = 50                    # maximal peptide length
    )

In [2]:
# from aiproteomics.modelgen import prosit1_gen

# model = prosit1_gen.build_prosit1_model()

2022-12-09 10:04:47.891487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-09 10:04:47.891724: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-09 10:04:47.891800: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-QCBKDU1): /proc/driver/nvidia/version does not exist
2022-12-09 10:04:47.893854: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 10:04:49.944587: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)
# graphviz not working, needs to be fixed

## Train the transformer model on our dataset

In [ ]:
learning_rate = aiproteomics.frag.models.CustomSchedule(512)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanAbsoluteError())

In [ ]:
history = model.fit(
    ds['train'],
    batch_size=1024,
    epochs=1,
    validation_data=ds['validate'],
    callbacks = []
)

## Check performance of model against validation data

In [ ]:
score_train = model.evaluate(ds['train'], verbose=0)
score_val = model.evaluate(ds['validate'], verbose=0)
score = model.evaluate(ds['test'], verbose=0)

print(score_train, score_val, "->", score)

## Prosit1 fragmentation model

In [ ]:
model = aiproteomics.frag.models.build_prosit1_model()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

## Fragmentation model

In [ ]:
model = aiproteomics.frag.models.build_frag_transformer_model(
        num_layers = 6,                 # number of layers
        d_model = 512,
        num_heads = 8,                  # Number of attention heads
        d_ff = 2048,                    # Hidden layer size in feed forward network inside transformer
        dropout_rate = 0.1,             #
        vocab_size = 22,                # number of aminoacids
        max_len = 50                    # maximal peptide length
    )

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)